In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv("C:\\Users\\Arnav Singla\\Downloads\\archive (9)\\amazon_delivery.csv")
print(df.head())
print(df.shape)

        Order_ID  Agent_Age  Agent_Rating  Store_Latitude  Store_Longitude  \
0  ialx566343618         37           4.9       22.745049        75.892471   
1  akqg208421122         34           4.5       12.913041        77.683237   
2  njpu434582536         23           4.4       12.914264        77.678400   
3  rjto796129700         38           4.7       11.003669        76.976494   
4  zguw716275638         32           4.6       12.972793        80.249982   

   Drop_Latitude  Drop_Longitude  Order_Date Order_Time Pickup_Time  \
0      22.765049       75.912471  19-03-2022   11:30:00    11:45:00   
1      13.043041       77.813237  25-03-2022   19:45:00    19:50:00   
2      12.924264       77.688400  19-03-2022   08:30:00    08:45:00   
3      11.053669       77.026494  05-04-2022   18:00:00    18:10:00   
4      13.012793       80.289982  26-03-2022   13:30:00    13:45:00   

      Weather  Traffic      Vehicle            Area  Delivery_Time  \
0       Sunny    High   motorcycle

In [2]:
df[df.select_dtypes(include='object').columns] = (
    df.select_dtypes(include='object')
      .apply(lambda x: x.str.strip())
)

C:\Users\Arnav Singla\AppData\Local\Temp\ipykernel_23572\610115705.py:2: Pandas4Warning: For backward compatibility, 'str' dtypes are included by select_dtypes when 'object' dtype is specified. This behavior is deprecated and will be removed in a future version. Explicitly pass 'str' to `include` to select them, or to `exclude` to remove them and silence this warning.
See https://pandas.pydata.org/docs/user_guide/migration-3-strings.html#string-migration-select-dtypes for details on how to write code that works with pandas 2 and 3.
  df.select_dtypes(include='object')
C:\Users\Arnav Singla\AppData\Local\Temp\ipykernel_23572\610115705.py:1: Pandas4Warning: For backward compatibility, 'str' dtypes are included by select_dtypes when 'object' dtype is specified. This behavior is deprecated and will be removed in a future version. Explicitly pass 'str' to `include` to select them, or to `exclude` to remove them and silence this warning.
See https://pandas.pydata.org/docs/user_guide/migratio

In [3]:
#count missing values
missing_values_count = df.isnull().sum()
print(missing_values_count[0:])

Order_ID            0
Agent_Age           0
Agent_Rating       54
Store_Latitude      0
Store_Longitude     0
Drop_Latitude       0
Drop_Longitude      0
Order_Date          0
Order_Time          0
Pickup_Time         0
Weather            91
Traffic             0
Vehicle             0
Area                0
Delivery_Time       0
Category            0
dtype: int64


In [4]:
df = df.dropna()

In [5]:
#drop useless columns
df = df.drop(columns=['Order_ID','Order_Date','Agent_Age','Order_Time','Pickup_Time'])

In [6]:
#create a new feature delivery_status using delivery time 
def categorise_time(t):
    if t<=110 :
        return "On-Time"
    elif t<= 170:
        return "Delayed"
    else:
        return "At risk"
df['Delivery_status'] = df['Delivery_Time'].apply(categorise_time)

In [7]:
print(df['Delivery_status'].value_counts())

Delivery_status
On-Time    18277
Delayed    17406
At risk     7911
Name: count, dtype: int64


In [8]:
#make a new feature -> distance , between store and drop coordinates
def distance(lat1, lon1, lat2, lon2):
    R = 6371  # Earth radius in km

    lat1, lon1, lat2, lon2 = map(
        np.radians, [lat1, lon1, lat2, lon2]
    )

    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = np.sin(dlat/2)**2 + \
        np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2

    c = 2 * np.arcsin(np.sqrt(a))
    return R * c
df['Distance']  = distance(
    df['Store_Latitude'],
    df['Store_Longitude'],
    df['Drop_Latitude'],
    df['Drop_Longitude']
)

In [9]:
df['Distance'] = df['Distance'].round()
print(df['Distance'].head(10))

0     3.0
1    20.0
2     2.0
3     8.0
4     6.0
5     5.0
6    17.0
7    20.0
8    20.0
9    10.0
Name: Distance, dtype: float64


In [10]:
#now drop coordinates columns
df = df.drop(['Store_Latitude','Store_Longitude',
              'Drop_Latitude','Drop_Longitude'],axis=1)

In [11]:
print(df['Weather'].value_counts())


Weather
Fog           7435
Stormy        7366
Cloudy        7277
Sandstorms    7238
Windy         7211
Sunny         7067
Name: count, dtype: int64


In [12]:
#encode weather column
df = pd.get_dummies(df, columns=['Weather'], drop_first=True)

In [13]:
#traffic column
print(df['Traffic'].value_counts())

Traffic
Low       14976
Jam       13710
Medium    10614
High       4294
Name: count, dtype: int64


In [14]:
#fill NaN values with other category 'Unknown'
df['Traffic'] = df['Traffic'].fillna('Unknown')
#one hot encode traffic
df = pd.get_dummies(df,columns=['Traffic'],drop_first=True)


In [15]:
#handle area column
print(df['Area'].value_counts())

Area
Metropolitian    32592
Urban             9715
Other             1135
Semi-Urban         152
Name: count, dtype: int64


In [16]:
df = pd.get_dummies(df,columns=['Area'],drop_first=True)

In [17]:
#handle Vehicle column
print(df['Vehicle'].value_counts())

Vehicle
motorcycle    25486
scooter       14590
van            3518
Name: count, dtype: int64


In [18]:
print(df.loc[df['Vehicle'] == 'bicycle', ['Delivery_status']])

Empty DataFrame
Columns: [Delivery_status]
Index: []


In [19]:
#one hot encode Vehicle
df = pd.get_dummies(df,columns=['Vehicle'],drop_first=True)

In [20]:
#handle Category
print(df['Category'].value_counts())

Category
Electronics     2841
Books           2813
Jewelry         2793
Toys            2771
Skincare        2764
Snacks          2762
Outdoors        2739
Apparel         2718
Sports          2708
Grocery         2687
Pet Supplies    2679
Home            2678
Cosmetics       2666
Kitchen         2663
Clothing        2657
Shoes           2655
Name: count, dtype: int64


In [21]:
#encode Category column
df = pd.get_dummies(df,columns=['Category'],drop_first=True)
df = df.drop(columns = 'Delivery_Time')

In [22]:
print(df.columns)

Index(['Agent_Rating', 'Delivery_status', 'Distance', 'Weather_Fog',
       'Weather_Sandstorms', 'Weather_Stormy', 'Weather_Sunny',
       'Weather_Windy', 'Traffic_Jam', 'Traffic_Low', 'Traffic_Medium',
       'Area_Other', 'Area_Semi-Urban', 'Area_Urban', 'Vehicle_scooter',
       'Vehicle_van', 'Category_Books', 'Category_Clothing',
       'Category_Cosmetics', 'Category_Electronics', 'Category_Grocery',
       'Category_Home', 'Category_Jewelry', 'Category_Kitchen',
       'Category_Outdoors', 'Category_Pet Supplies', 'Category_Shoes',
       'Category_Skincare', 'Category_Snacks', 'Category_Sports',
       'Category_Toys'],
      dtype='str')


In [23]:
print(df.head())

   Agent_Rating Delivery_status  Distance  Weather_Fog  Weather_Sandstorms  \
0           4.9         Delayed       3.0        False               False   
1           4.5         Delayed      20.0        False               False   
2           4.4         Delayed       2.0        False                True   
3           4.7         On-Time       8.0        False               False   
4           4.6         Delayed       6.0        False               False   

   Weather_Stormy  Weather_Sunny  Weather_Windy  Traffic_Jam  Traffic_Low  \
0           False           True          False        False        False   
1            True          False          False         True        False   
2           False          False          False        False         True   
3           False           True          False        False        False   
4           False          False          False        False        False   

   ...  Category_Home  Category_Jewelry  Category_Kitchen  Category_

In [24]:
#define features and target
X = df.drop(columns=['Delivery_status'])
y = df['Delivery_status']


In [25]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y = le.fit_transform(y)


In [26]:
#train test split 
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2
                                                 ,random_state=42,stratify=y)

In [27]:
#random forest model
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(
    n_estimators=200,
    random_state=42,
    class_weight='balanced',
)


In [28]:
model.fit(X_train,y_train)
y_pred = model.predict(X_test)

In [29]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred, target_names=le.classes_))


              precision    recall  f1-score   support

     At risk       0.69      0.69      0.69      1582
     Delayed       0.60      0.59      0.60      3481
     On-Time       0.72      0.72      0.72      3656

    accuracy                           0.66      8719
   macro avg       0.67      0.67      0.67      8719
weighted avg       0.66      0.66      0.66      8719



In [30]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_pred)
pd.DataFrame(cm, index=le.classes_, columns=le.classes_)


,At risk,Delayed,On-Time
At risk,1085,427,70
Delayed,440,2071,970
On-Time,50,980,2626


In [31]:
#check which feature impacts most 
feature_importance = pd.Series(
    model.feature_importances_,
    index=X.columns
).sort_values(ascending=False)

print(feature_importance.head(10))


Distance              0.245875
Agent_Rating          0.235139
Category_Grocery      0.058077
Traffic_Low           0.055914
Weather_Sunny         0.043189
Traffic_Jam           0.035108
Vehicle_scooter       0.028340
Weather_Sandstorms    0.023456
Weather_Stormy        0.022770
Weather_Windy         0.022213
dtype: float64
